In [3]:
# get canonical pts
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import pickle

model_dir = '../../models/'
dae_dir = 'visual_meshes/visual.dae'
png_dir = 'visual_meshes/texture_map.png'

### Get id-name pairs

In [69]:
import pandas as pd
csv_dir = '../../training_data/objects_v1.csv'
obj_data = pd.read_csv(csv_dir,sep=',')
id2name = list(obj_data['object']) # begins at 0
id2symm = list(obj_data['geometric_symmetry'])

### Load mesh and sample point cloud

In [5]:
import trimesh
from trimesh import viewer
from trimesh import sample

def FPS(ptcloud,count):
    U = ptcloud
    S = []
    dist_to_S = np.full(len(U),np.inf)
    
    newpt_idx = 0

    for _ in range(count):
        S.append(U[newpt_idx])
        dist_offer = np.sum((U - U[newpt_idx])**2,axis=1)
        dist_to_S = np.min(np.stack((dist_offer,dist_to_S)),axis=0)
        # find new point
        newpt_idx = np.argmax(dist_to_S)
    
    return np.array(S)

def get_canonical_pcd(obj_id,count):
    obj_name = id2name[obj_id]

    dae_f = open(os.path.join(model_dir+obj_name,dae_dir),'rb')
    png_resolver = trimesh.resolvers.FilePathResolver(os.path.join(model_dir+obj_name,png_dir))
    scenemesh = trimesh.exchange.dae.load_collada(dae_f,png_resolver)
    geo = list(scenemesh['geometry'].values())[0]
    mesh = trimesh.Trimesh(geo['vertices'],geo['faces'],vertex_normals=geo['vertex_normals'])
    sample_pcd,_ = sample.sample_surface(mesh, 10*count, face_weight=None, sample_color=False)
    pt_cloud_fps = FPS(sample_pcd,count)
    return pt_cloud_fps


In [6]:
import open3d
def showpcd(pcd):
    points = open3d.utility.Vector3dVector(pcd)
    _pcd = open3d.geometry.PointCloud()
    _pcd.points = points
    open3d.visualization.draw_geometries([_pcd])
# showpcd(get_canonical_pcd(2,10000))

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [7]:
from tqdm import tqdm
NUM_OBJECTS = 79
canonical_pcds_filename = 'canonical_pcds.pkl'
if os.path.exists(canonical_pcds_filename):
    with open(canonical_pcds_filename, 'rb') as f:
        canonical_pcds = pickle.load(f)
else:
    canonical_pcds = []
    for id in tqdm(range(NUM_OBJECTS)):
        canonical_pcds.append(get_canonical_pcd(id,5000))
    with open(canonical_pcds_filename, 'wb') as f:
        pickle.dump(canonical_pcds, f)

### Load data

In [8]:
def load_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

training_data_dir = "../../training_data/v2.2"
split_dir = "../../training_data/splits/v2"

def get_scene(scenename):
    # scenename refers to the i-j-k prefix
    prefix = os.path.join(training_data_dir, scenename)
    rgb = [prefix + "_color_kinect.png"]
    depth = [prefix + "_depth_kinect.png"]
    label = [prefix + "_label_kinect.png"]
    meta = [prefix + "_meta.pkl"]
    return rgb, depth, label, meta

def get_split_files(split_name):
    with open(os.path.join(split_dir, f"{split_name}.txt"), 'r') as f:
        prefix = [os.path.join(training_data_dir, line.strip()) for line in f if line.strip() and int(line.strip()[0]) <= 2]
        rgb = [p + "_color_kinect.png" for p in prefix]
        depth = [p + "_depth_kinect.png" for p in prefix]
        label = [p + "_label_kinect.png" for p in prefix]
        meta = [p + "_meta.pkl" for p in prefix]
    return rgb, depth, label, meta

In [9]:
def lift_seg(depth,label,meta):
    # lifting
    intrinsic = meta['intrinsic']
    z = depth
    v, u = np.indices(z.shape)
    uv1 = np.stack([u + 0.5, v + 0.5, np.ones_like(z)], axis=-1)
    scene_pcd = uv1 @ np.linalg.inv(intrinsic).T * z[..., None]  # [H, W, 3]
    # segmenting
    pcds = {}
    scales = {}
    for id in meta['object_ids']:
        raw_pcd = scene_pcd[np.where(label==id)]
        scales[id] = meta['scales'][id]
        pcds[id] = (raw_pcd) / meta['scales'][id]
        
    return pcds, scales


In [41]:
import copy
def draw_registration_result(source, target, transformation=None, show_normal=False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    if transformation is not None:
        source_temp.transform(transformation)
    open3d.visualization.draw_geometries([source_temp, target_temp],point_show_normal=show_normal)

In [42]:
def get_open3d_PointCloud(pcd_nparray):
    points = open3d.utility.Vector3dVector(pcd_nparray.reshape([-1, 3]))
    pcd = open3d.geometry.PointCloud()
    pcd.points = points
    pcd.estimate_normals(search_param=open3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    return pcd

In [66]:
def scene_to_pcds(scenename):
    rgb_files, depth_files, label_files, meta_files = get_scene(scenename)
    rgb = np.array(Image.open(rgb_files[0])) / 255
    depth = np.array(Image.open(depth_files[0])) / 1000
    label = np.array(Image.open(label_files[0]))
    meta = load_pickle(meta_files[0])

    poses = np.array([meta['extrinsic']@meta['poses_world'][idx] for idx in meta['object_ids']]) # ground truth

    pcds, scales = lift_seg(depth,label,meta)

    return meta['object_ids'], pcds, scales, poses

#name = '1-1-1'
#ids,pcds,scales,poses = scene_to_pcds(name)
#_, _, _, meta_files = get_scene(name)
#meta = load_pickle(meta_files[0])
#k = 1
#draw_registration_result(get_open3d_PointCloud(canonical_pcds[ids[k]]),get_open3d_PointCloud(pcds[ids[k]]))
#draw_registration_result(get_open3d_PointCloud(canonical_pcds[ids[k]]),get_open3d_PointCloud(pcds[ids[k]]),meta['poses_world'][ids[k]])
#draw_registration_result(get_open3d_PointCloud(canonical_pcds[ids[k]]),get_open3d_PointCloud(pcds[ids[k]]),poses[k])
#poses[k][:3,3] = 2*poses[k][:3,3] 
#draw_registration_result(get_open3d_PointCloud(canonical_pcds[ids[k]]),get_open3d_PointCloud(pcds[ids[k]]),poses[k])

**An awful mistake in ground-truth data: some transation vector lost a factor of 2...**

### ICP

### init estimation

In [20]:
def preprocess_point_cloud(pcd, voxel_size):
    #print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_feature = voxel_size * 5
    #print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = open3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        open3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [18]:
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    #print(":: RANSAC registration on downsampled point clouds.")
    #print("   Since the downsampling voxel size is %.3f," % voxel_size)
    #print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = open3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        open3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            open3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            open3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], open3d.pipelines.registration.RANSACConvergenceCriteria(10000000, 0.999999))
    return result

### eval

In [ ]:
def compare(T,T_pred,scale,symm):
    t,t_pred = T[:3,3], T_pred[:3,3]
    # see above
    t_error = min(np.linalg.norm((t-t_pred)*scale),np.linalg.norm((2*t-t_pred)*scale))
    R,R_pred = T[:3,:3], T_pred[:3,:3]

    cos_R_error = np.clip(0.5*(np.trace(R@R_pred.T)-1),-1,1)

    if 'inf' in symm:
        R_error = 0
    elif symm=='no':
        R_error = np.arccos(cos_R_error) *180 / np.pi
    elif '4' in symm:
        R_error = (np.arccos(cos_R_error) *180 / np.pi)%90
        R_error = min(abs(R_error),abs(90-R_error))
    elif '2' in symm:
        R_error = (np.arccos(cos_R_error) *180 / np.pi)%180
        R_error = min(abs(R_error),abs(180-R_error))
    return t_error,R_error

### Execution

In [72]:
voxel_size = 0.005
correct_cnt = 0
total_cnt = 0
with open(os.path.join(split_dir, "val.txt"), 'r') as f:
    scenenames = [line.strip() for line in f if line.strip()]
for scenename in tqdm(scenenames):
    ids, pcds, scales, poses = scene_to_pcds(scenename)
    for i,id in enumerate(ids):
        source = get_open3d_PointCloud(canonical_pcds[id])
        target = get_open3d_PointCloud(pcds[id])
        #draw_registration_result(source,target,show_normal=False)

        source_down, source_fpfh = preprocess_point_cloud(source,voxel_size)
        target_down, target_fpfh = preprocess_point_cloud(target,voxel_size)
        fpfh_init = execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)
        #draw_registration_result(source_down, target_down, fpfh_init.transformation)
        
        reg_refine = open3d.pipelines.registration.registration_icp(
            source, target, 2*voxel_size, fpfh_init.transformation, 
            open3d.pipelines.registration.TransformationEstimationPointToPlane(), 
            open3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=10000))
        
        dt,dtheta=compare(poses[i],reg_refine.transformation,scales[id],id2symm[id])
        if abs(dt)<0.01 and dtheta<5:
            correct_cnt+=1
        total_cnt+=1
print(f'Accuracy = {correct_cnt/total_cnt}')
        

  1%|          | 29/2800 [02:51<1:53:29,  2.46s/it] C:\Users\不灭的~1\AppData\Local\Temp/ipykernel_43544/3211697845.py:15: RuntimeWarning: invalid value encountered in arccos
  R_error = (np.arccos(0.5*(np.trace(R@R_pred.T)-1)) *180 / np.pi)%180
C:\Users\不灭的~1\AppData\Local\Temp/ipykernel_43544/3211697845.py:15: RuntimeWarning: invalid value encountered in double_scalars
  R_error = (np.arccos(0.5*(np.trace(R@R_pred.T)-1)) *180 / np.pi)%180
  1%|          | 34/2800 [03:03<1:47:25,  2.33s/it]C:\Users\不灭的~1\AppData\Local\Temp/ipykernel_43544/3211697845.py:12: RuntimeWarning: invalid value encountered in arccos
  R_error = (np.arccos(0.5*(np.trace(R@R_pred.T)-1)) *180 / np.pi)%90
C:\Users\不灭的~1\AppData\Local\Temp/ipykernel_43544/3211697845.py:12: RuntimeWarning: invalid value encountered in double_scalars
  R_error = (np.arccos(0.5*(np.trace(R@R_pred.T)-1)) *180 / np.pi)%90
 20%|█▉        | 550/2800 [46:45<4:09:39,  6.66s/it] 

: 

: 